In [6]:
# Sempre rode isso para começar o Notebook
# De modo resumido, isso faz com que o notebook possa acessar o Django

import sys
sys.path.append("../../")

import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serase_projeto.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [7]:
# Digite seu código aqui!

In [3]:
# Testando values_list no PadraoMovimentacao

from serase_app.models import PadraoMovimentacao
from django.db.models import F

lista = PadraoMovimentacao.objects.values("id", "descricao", "periodo", "dia_cobranca", "data_inicio", "data_fim", valor_padrao=F("valor"), categoria=F("cod_categoria__nome"), tipo=F("receita_despesa"))
lista = list(lista)
lista

[{'id': 1,
  'descricao': 'Spotify',
  'periodo': 1,
  'dia_cobranca': 1,
  'data_inicio': datetime.date(2018, 4, 18),
  'data_fim': None,
  'valor_padrao': Decimal('-26.90'),
  'categoria': 'Lazer',
  'tipo': 'D'},
 {'id': 2,
  'descricao': 'Bolsa IC',
  'periodo': 1,
  'dia_cobranca': 6,
  'data_inicio': datetime.date(2020, 7, 8),
  'data_fim': datetime.date(2021, 7, 8),
  'valor_padrao': Decimal('300.00'),
  'categoria': 'Sálario',
  'tipo': 'R'},
 {'id': 3,
  'descricao': 'Conta de luz',
  'periodo': 1,
  'dia_cobranca': 6,
  'data_inicio': None,
  'data_fim': None,
  'valor_padrao': None,
  'categoria': 'Habitação',
  'tipo': 'D'},
 {'id': 4,
  'descricao': 'Internet',
  'periodo': 1,
  'dia_cobranca': 5,
  'data_inicio': datetime.date(2020, 2, 4),
  'data_fim': None,
  'valor_padrao': Decimal('-59.90'),
  'categoria': 'Habitação',
  'tipo': 'D'},
 {'id': 5,
  'descricao': 'Sálario',
  'periodo': 1,
  'dia_cobranca': 5,
  'data_inicio': None,
  'data_fim': None,
  'valor_padrao': 

In [6]:
# Testando o choices do PadraoMovimentacao

#from serase_app.models import PadraoMovimentacao
from django.db.models import F, Case, When, CharField, Value

# > Consulta

# Pego de https://stackoverflow.com/a/60158573
class WithChoices(Case):
    def __init__(self, model, field, condition=None, then=None, **lookups):
        choices = dict(model._meta.get_field(field).flatchoices)
        whens = [When(**{field: k, 'then': Value(v)}) for k, v in choices.items()]
        return super().__init__(*whens, output_field=CharField())


PadraoMovimentacao.objects.values()

<QuerySet [{'id': 1, 'receita_despesa': 'D', 'descricao': 'Spotify', 'periodo': 1, 'valor': Decimal('-26.90'), 'dia_cobranca': 1, 'data_inicio': datetime.date(2018, 4, 18), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 4}, {'id': 2, 'receita_despesa': 'R', 'descricao': 'Bolsa IC', 'periodo': 1, 'valor': Decimal('300.00'), 'dia_cobranca': 6, 'data_inicio': datetime.date(2020, 7, 8), 'data_fim': datetime.date(2021, 7, 8), 'cod_usuario_id': 1, 'cod_categoria_id': 9}, {'id': 3, 'receita_despesa': 'D', 'descricao': 'Conta de luz', 'periodo': 1, 'valor': None, 'dia_cobranca': 6, 'data_inicio': None, 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 4, 'receita_despesa': 'D', 'descricao': 'Internet', 'periodo': 1, 'valor': Decimal('-59.90'), 'dia_cobranca': 5, 'data_inicio': datetime.date(2020, 2, 4), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 5, 'receita_despesa': 'Emma', 'descricao': 'Sálario', 'periodo': 1, 'valor': Decimal('235

In [9]:
# Testando movimentacoes

from serase_app.models import Movimentacao

In [27]:
valores = Movimentacao.objects.filter(valor_pago__isnull=False).values_list("descricao", "data_lancamento").order_by("-data_lancamento")

for nom, dat in valores:
    print(nom + " " + str(dat))

Mate coro e sanduba 2020-11-01
bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03
Milkshake McDonnadls 2020-09-15
Compras 2020-09-12
Bolsa ic 2020-09-08
Conta de luz 2020-09-08
Internet 2020-09-08
Internet 2020-09-08
Spotify 2020-09-08
Sálario 2020-09-07
Viola 2020-09-02
Bolsa ic 2020-08-10
Conta de luz 2020-08-10
Internet 2020-08-10
Spotify 2020-08-10
Sálario 2020-08-07
Adoção do Cumpa, o Cacto 2020-07-16
Outer wilds 2020-07-10


datetime.datetime(2020, 11, 20, 15, 29, 33, 750615)

In [28]:
# Teste datetime: objeto data apartir de string e filtrar por data

from datetime import datetime

data = datetime.strptime("2020-10-26", '%Y-%m-%d').date()
Movimentacao.objects.filter(data_lancamento__gte=data)

<QuerySet [<Movimentacao: Mate coro e sanduba>, <Movimentacao: bebida Halloween>, <Movimentacao: Make pro Halloween>, <Movimentacao: Dinheiro do Spotify>]>

In [5]:
a = Movimentacao.objects.filter(descricao="Churros").values()
list(a)[0]

{'id': 13,
 'descricao': 'Churros',
 'valor_esperado': None,
 'valor_pago': Decimal('-5.00'),
 'data_geracao': None,
 'data_lancamento': datetime.date(2020, 10, 3),
 'cod_usuario_id': 1,
 'cod_categoria_id': 5,
 'cod_padrao_id': None}

In [19]:
from datetime import datetime

mes_ano = datetime.strptime("2020-10", '%Y-%m')

a = Movimentacao.objects.filter(data_lancamento__month=mes_ano.month, data_lancamento__year=mes_ano.year).order_by("-data_lancamento")
for q in a:
    print(q.descricao, q.data_lancamento)

bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03


In [21]:
datetime.today().year

2020

In [27]:
from django.db.models import Sum

mes_ano = datetime.strptime("2020-10", '%Y-%m')
query_movimentacao = Movimentacao.objects.filter(valor_pago__isnull=False)
query_movimentacao = query_movimentacao.filter(data_lancamento__year=mes_ano.year, data_lancamento__month=mes_ano.month)
query_movimentacao.aggregate(Sum("valor_pago"))["valor_pago__sum"]

Decimal('2126.5')